In [1]:
library(tidyr)
library(car)
library(emmeans)

# increase the number of characters that can be printed
options("width"=200)

Loading required package: carData



In [2]:
df <- read.csv("data/data.csv")

# only consider the cases with one face
df  <- subset(df, df$has_faces == 1)

# Face-ism in the person queries


In [3]:
# filter by the query type person, i.e. "woman", man", "intelligent woman" and "intelligent man" queries
dfg <- subset(df, df$query_type=="man" | df$query_type=="woman")

# create gender variable based on the face
dfg$gender <- NA
dfg[which(dfg$has_faces_male==1),"gender"]<-'Male'
dfg[which(dfg$has_faces_female==1),"gender"]<-'Female'

# ignore the cases in which the query and gender does not match
dfg$gendermatch <- 0
dfg[which(dfg$has_faces_male==1 & dfg$query_type=="man"),"gendermatch"]<-1
dfg[which(dfg$has_faces_female==1 & dfg$query_type=="woman"),"gendermatch"]<-1
dfg <- subset(dfg, dfg$gendermatch == 1)



## 1. Anova table

In [4]:
# fit the model
fit <- lm(faceism ~ gender*query_adjective*engine+region+browser+wave, data=dfg)

# display general anova table (factors and interactions only)
print(Anova(fit, type="III"))

Anova Table (Type III tests)

Response: faceism
                              Sum Sq    Df  F value    Pr(>F)    
(Intercept)                    28.33     1 786.7598 < 2.2e-16 ***
gender                          0.20     1   5.6621 0.0173482 *  
query_adjective                 0.34     1   9.4237 0.0021459 ** 
engine                          1.61     3  14.9430  1.04e-09 ***
region                          0.02     2   0.2722 0.7617320    
browser                         0.00     1   0.1087 0.7416694    
wave                            0.01     1   0.4113 0.5213100    
gender:query_adjective          0.40     1  11.1585 0.0008386 ***
gender:engine                   5.39     3  49.9234 < 2.2e-16 ***
query_adjective:engine          4.43     3  41.0241 < 2.2e-16 ***
gender:query_adjective:engine   0.98     3   9.0692  5.39e-06 ***
Residuals                     517.31 14366                       
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1


## 2. Contrast tests

### 2.1 Accross search engines

In [5]:
# full table of contrasts by search engine including. The interactions include those across 
# queries, but we only report the ones within the query term (i.e. "person Female / person Male" 
# and intelligent person Female / person Male")
pairs_engine <- pairs(emmeans(fit, ~ query_adjective * gender | engine),type = "response")

# print the pairs
print(pairs_engine)

engine = Baidu:
 contrast                                 estimate      SE    df t.ratio p.value
 intelligent Female - no adjective Female  0.04067 0.01325 14366   3.070  0.0115
 intelligent Female - intelligent Male     0.04193 0.01762 14366   2.380  0.0811
 intelligent Female - no adjective Male    0.01249 0.01468 14366   0.851  0.8301
 no adjective Female - intelligent Male    0.00126 0.01500 14366   0.084  0.9998
 no adjective Female - no adjective Male  -0.02818 0.01140 14366  -2.472  0.0644
 intelligent Male - no adjective Male     -0.02944 0.01628 14366  -1.809  0.2692

engine = Bing:
 contrast                                 estimate      SE    df t.ratio p.value
 intelligent Female - no adjective Female  0.07840 0.00794 14366   9.875  <.0001
 intelligent Female - intelligent Male    -0.10682 0.00858 14366 -12.457  <.0001
 intelligent Female - no adjective Male    0.00165 0.00783 14366   0.211  0.9967
 no adjective Female - intelligent Male   -0.18522 0.00803 14366 -23.076  <.0

### 2.2 Accross interactions

In [6]:
# full table of contrasts by interactions of search engine and query
# we only report the ones within the query term (i.e. "person Female / person Mal" and
# intelligent person Female / person Male")
pairs_interaction <- pairs(emmeans(fit, ~ query_adjective * gender  | engine,type = "response"), interaction = "pairwise")

# print the pairs
print(pairs_interaction)

engine = Baidu:
 query_adjective_pairwise   gender_pairwise estimate     SE    df t.ratio p.value
 intelligent - no adjective Female - Male     0.0701 0.0210 14366   3.340  0.0008

engine = Bing:
 query_adjective_pairwise   gender_pairwise estimate     SE    df t.ratio p.value
 intelligent - no adjective Female - Male    -0.0301 0.0112 14366  -2.682  0.0073

engine = Google:
 query_adjective_pairwise   gender_pairwise estimate     SE    df t.ratio p.value
 intelligent - no adjective Female - Male     0.0290 0.0130 14366   2.230  0.0258

engine = Yandex:
 query_adjective_pairwise   gender_pairwise estimate     SE    df t.ratio p.value
 intelligent - no adjective Female - Male    -0.0236 0.0115 14366  -2.048  0.0406

Results are averaged over the levels of: region, browser, wave 


## 3 Confidence intervals

### 3.1 Accross search engines

In [7]:
# the confidence intervals here correspond to the ones in section 2.1
confint(pairs_engine)[,c(1,2,6,7)]

,contrast,engine,lower.CL,upper.CL
,<fct>,<fct>,<dbl>,<dbl>
1,intelligent Female - no adjective Female,Baidu,0.006630233,0.074706134
2,intelligent Female - intelligent Male,Baidu,-0.003344773,0.087206273
3,intelligent Female - no adjective Male,Baidu,-0.025224077,0.050197052
4,no adjective Female - intelligent Male,Baidu,-0.037284021,0.039809155
5,no adjective Female - no adjective Male,Baidu,-0.057477165,0.001113774
6,intelligent Male - no adjective Male,Baidu,-0.071270645,0.012382120
7,intelligent Female - no adjective Female,Bing,0.057997726,0.098793428
8,intelligent Female - intelligent Male,Bing,-0.128853771,-0.084787719
9,intelligent Female - no adjective Male,Bing,-0.018465155,0.021768316


### 3.2 Accross interactions

In [8]:
# the confidence intervals here correspond to the ones in section 2.2
confint(pairs_interaction)[,c(1,2,3,7,8)]

,query_adjective_pairwise,gender_pairwise,engine,lower.CL,upper.CL
,<fct>,<fct>,<fct>,<dbl>,<dbl>
1,intelligent - no adjective,Female - Male,Baidu,0.028971290,0.111253602
2,intelligent - no adjective,Female - Male,Bing,-0.052059072,-0.008094426
3,intelligent - no adjective,Female - Male,Google,0.003508066,0.054431349
4,intelligent - no adjective,Female - Male,Yandex,-0.046137156,-0.001009237


## 2.3 Effect sizes

### 3.1 Accross search engines

In [9]:
# the effect sizess here correspond to the ones in section 2.1
summary(eff_size(pairs_engine, sigma = sigma(fit), edf = df.residual(fit), method = "identity"))[,c(1,2,3,6,7)]

,contrast,engine,effect.size,lower.CL,upper.CL
,<fct>,<fct>,<dbl>,<dbl>,<dbl>
1,(intelligent Female - no adjective Female),Baidu,0.214312012,0.07744754,0.35117648
2,(intelligent Female - intelligent Male),Baidu,0.220965451,0.03892731,0.40300359
3,(intelligent Female - no adjective Male),Baidu,0.065800931,-0.08580794,0.21740980
4,(no adjective Female - intelligent Male),Baidu,0.006653439,-0.14831459,0.16162147
5,(no adjective Female - no adjective Male),Baidu,-0.148511081,-0.26629954,-0.03072262
6,(intelligent Male - no adjective Male),Baidu,-0.155164520,-0.32332779,0.01299875
7,(intelligent Female - no adjective Female),Bing,0.413126736,0.33098266,0.49527081
8,(intelligent Female - intelligent Male),Bing,-0.562920863,-0.65173863,-0.47410310
9,(intelligent Female - no adjective Male),Bing,0.008703453,-0.07217148,0.08957839


### 3.2 Accross interactions

In [10]:
# the effect sizess here correspond to the ones in section 2.2
summary( eff_size(pairs_interaction, sigma(fit), df.residual(fit), method = "identity"))[,c(1,2,3,6,7)]


,contrast,engine,effect.size,lower.CL,upper.CL
,<fct>,<fct>,<dbl>,<dbl>,<dbl>
1,(intelligent - no adjective Female - Male),Baidu,0.3694765,0.15262997,0.586323096
2,(intelligent - no adjective Female - Male),Bing,-0.1584976,-0.27435389,-0.042641273
3,(intelligent - no adjective Female - Male),Google,0.1526637,0.01847509,0.286852359
4,(intelligent - no adjective Female - Male),Yandex,-0.1242253,-0.24314092,-0.005309773
